In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from metrics import *
from Datagens import *
from ResidualUnet import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_res_unet(n_channels=3, ker_init='he_normal')
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  2320        ['conv2d[0][0]']                 
                                )                                                             

                                                                                                  
 up_sampling2d_2 (UpSampling2D)  (None, 64, 64, 64)  0           ['add_5[0][0]']                  
                                                                                                  
 concatenate_2 (Concatenate)    (None, 64, 64, 96)   0           ['up_sampling2d_2[0][0]',        
                                                                  'add[0][0]']                    
                                                                                                  
 conv2d_20 (Conv2D)             (None, 64, 64, 32)   27680       ['concatenate_2[0][0]']          
                                                                                                  
 conv2d_22 (Conv2D)             (None, 64, 64, 32)   3104        ['concatenate_2[0][0]']          
                                                                                                  
 conv2d_21

In [3]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

In [4]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [5]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [6]:
csv_logger = CSVLogger('Logs/training/resU_net_2d.log', separator=',', append=False)

eval_csv_logger = CSVLogger('Logs/testing/resU_net_2d.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [7]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/resU_net_2d.h5")

Epoch 1/100
200/200 [==============================] - 60s 271ms/step - loss: 0.0930 - accuracy: 0.9820 - mean_io_u_1: 0.5951 - dice_coef: 0.2709 - dice_coef_necrotic: 0.0442 - dice_coef_edema: 0.1284 - dice_coef_enhancing: 0.0604 - val_loss: 0.0746 - val_accuracy: 0.9816 - val_mean_io_u_1: 0.6466 - val_dice_coef: 0.2783 - val_dice_coef_necrotic: 0.0434 - val_dice_coef_edema: 0.0997 - val_dice_coef_enhancing: 0.0446 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 52s 262ms/step - loss: 0.0650 - accuracy: 0.9834 - mean_io_u_1: 0.5474 - dice_coef: 0.2912 - dice_coef_necrotic: 0.0962 - dice_coef_edema: 0.1855 - dice_coef_enhancing: 0.0917 - val_loss: 0.0686 - val_accuracy: 0.9816 - val_mean_io_u_1: 0.6327 - val_dice_coef: 0.2959 - val_dice_coef_necrotic: 0.1050 - val_dice_coef_edema: 0.2505 - val_dice_coef_enhancing: 0.1021 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0515 - accuracy: 0.9845 - mean_io_u_1: 0.6374 - dice_c

Epoch 20/100
200/200 [==============================] - ETA: 0s - loss: 0.0176 - accuracy: 0.9938 - mean_io_u_1: 0.7247 - dice_coef: 0.5998 - dice_coef_necrotic: 0.5415 - dice_coef_edema: 0.7294 - dice_coef_enhancing: 0.6785
Epoch 20: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
200/200 [==============================] - 53s 266ms/step - loss: 0.0176 - accuracy: 0.9938 - mean_io_u_1: 0.7247 - dice_coef: 0.5998 - dice_coef_necrotic: 0.5415 - dice_coef_edema: 0.7294 - dice_coef_enhancing: 0.6785 - val_loss: 0.0287 - val_accuracy: 0.9916 - val_mean_io_u_1: 0.7766 - val_dice_coef: 0.5833 - val_dice_coef_necrotic: 0.4737 - val_dice_coef_edema: 0.7047 - val_dice_coef_enhancing: 0.6084 - lr: 0.0010
Epoch 21/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0151 - accuracy: 0.9946 - mean_io_u_1: 0.7795 - dice_coef: 0.6323 - dice_coef_necrotic: 0.5824 - dice_coef_edema: 0.7641 - dice_coef_enhancing: 0.7123 - val_loss: 0.0235 - val_accuracy: 0.9924 - 

200/200 [==============================] - 52s 261ms/step - loss: 0.0123 - accuracy: 0.9956 - mean_io_u_1: 0.8176 - dice_coef: 0.6784 - dice_coef_necrotic: 0.6520 - dice_coef_edema: 0.8131 - dice_coef_enhancing: 0.7628 - val_loss: 0.0241 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8273 - val_dice_coef: 0.6337 - val_dice_coef_necrotic: 0.5394 - val_dice_coef_edema: 0.7491 - val_dice_coef_enhancing: 0.6838 - lr: 1.0000e-06
Epoch 36/100
200/200 [==============================] - 53s 263ms/step - loss: 0.0123 - accuracy: 0.9956 - mean_io_u_1: 0.8175 - dice_coef: 0.6785 - dice_coef_necrotic: 0.6522 - dice_coef_edema: 0.8133 - dice_coef_enhancing: 0.7628 - val_loss: 0.0241 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8274 - val_dice_coef: 0.6337 - val_dice_coef_necrotic: 0.5393 - val_dice_coef_edema: 0.7492 - val_dice_coef_enhancing: 0.6836 - lr: 1.0000e-06
Epoch 37/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0123 - accuracy: 0.9956 - mean_io_u_1: 0.8181 - dice_coe

Epoch 54/100
200/200 [==============================] - 53s 266ms/step - loss: 0.0122 - accuracy: 0.9956 - mean_io_u_1: 0.8192 - dice_coef: 0.6803 - dice_coef_necrotic: 0.6540 - dice_coef_edema: 0.8146 - dice_coef_enhancing: 0.7642 - val_loss: 0.0241 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8285 - val_dice_coef: 0.6346 - val_dice_coef_necrotic: 0.5397 - val_dice_coef_edema: 0.7494 - val_dice_coef_enhancing: 0.6843 - lr: 1.0000e-06
Epoch 55/100
200/200 [==============================] - 53s 266ms/step - loss: 0.0122 - accuracy: 0.9956 - mean_io_u_1: 0.8189 - dice_coef: 0.6805 - dice_coef_necrotic: 0.6543 - dice_coef_edema: 0.8151 - dice_coef_enhancing: 0.7642 - val_loss: 0.0241 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8288 - val_dice_coef: 0.6346 - val_dice_coef_necrotic: 0.5396 - val_dice_coef_edema: 0.7492 - val_dice_coef_enhancing: 0.6842 - lr: 1.0000e-06
Epoch 56/100
200/200 [==============================] - 54s 268ms/step - loss: 0.0122 - accuracy: 0.9956 - mean_io_u_1: 0.81

Epoch 73/100
200/200 [==============================] - 53s 262ms/step - loss: 0.0121 - accuracy: 0.9956 - mean_io_u_1: 0.8212 - dice_coef: 0.6822 - dice_coef_necrotic: 0.6563 - dice_coef_edema: 0.8164 - dice_coef_enhancing: 0.7653 - val_loss: 0.0241 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8303 - val_dice_coef: 0.6354 - val_dice_coef_necrotic: 0.5403 - val_dice_coef_edema: 0.7494 - val_dice_coef_enhancing: 0.6849 - lr: 1.0000e-06
Epoch 74/100
200/200 [==============================] - 52s 262ms/step - loss: 0.0121 - accuracy: 0.9956 - mean_io_u_1: 0.8213 - dice_coef: 0.6822 - dice_coef_necrotic: 0.6564 - dice_coef_edema: 0.8165 - dice_coef_enhancing: 0.7656 - val_loss: 0.0241 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8306 - val_dice_coef: 0.6355 - val_dice_coef_necrotic: 0.5403 - val_dice_coef_edema: 0.7494 - val_dice_coef_enhancing: 0.6849 - lr: 1.0000e-06
Epoch 75/100
200/200 [==============================] - 52s 262ms/step - loss: 0.0121 - accuracy: 0.9956 - mean_io_u_1: 0.82

Epoch 92/100
200/200 [==============================] - 52s 261ms/step - loss: 0.0120 - accuracy: 0.9957 - mean_io_u_1: 0.8230 - dice_coef: 0.6838 - dice_coef_necrotic: 0.6585 - dice_coef_edema: 0.8182 - dice_coef_enhancing: 0.7669 - val_loss: 0.0242 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8325 - val_dice_coef: 0.6363 - val_dice_coef_necrotic: 0.5406 - val_dice_coef_edema: 0.7494 - val_dice_coef_enhancing: 0.6854 - lr: 1.0000e-06
Epoch 93/100
200/200 [==============================] - 52s 262ms/step - loss: 0.0120 - accuracy: 0.9957 - mean_io_u_1: 0.8232 - dice_coef: 0.6839 - dice_coef_necrotic: 0.6585 - dice_coef_edema: 0.8181 - dice_coef_enhancing: 0.7669 - val_loss: 0.0242 - val_accuracy: 0.9927 - val_mean_io_u_1: 0.8325 - val_dice_coef: 0.6363 - val_dice_coef_necrotic: 0.5407 - val_dice_coef_edema: 0.7494 - val_dice_coef_enhancing: 0.6853 - lr: 1.0000e-06
Epoch 94/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0120 - accuracy: 0.9957 - mean_io_u_1: 0.82

In [ ]:
# model = keras.models.load_model('Models/U_net_200_2d_3ch.h5', 
#                                    custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
#                                                    "dice_coef": dice_coef,
#                                                    "precision": precision,
#                                                    "sensitivity":sensitivity,
#                                                    "specificity":specificity,
#                                                    "dice_coef_necrotic": dice_coef_necrotic,
#                                                    "dice_coef_edema": dice_coef_edema,
#                                                    "dice_coef_enhancing": dice_coef_enhancing
#                                                   }, compile=True)

In [8]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

36/36 [==============================] - 8s 208ms/step - loss: 0.0207 - accuracy: 0.9935 - mean_io_u_1: 0.8338 - dice_coef: 0.6793 - dice_coef_necrotic: 0.6072 - dice_coef_edema: 0.7875 - dice_coef_enhancing: 0.7623


[0.02069905586540699,
 0.9935384392738342,
 0.8338388204574585,
 0.6793285012245178,
 0.6072157621383667,
 0.7874513268470764,
 0.762266993522644]

In [9]:
valid_csv_logger = CSVLogger('Logs/validation/resU_net_2d.log', separator=',', append=False)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)

60/60 [==============================] - 12s 195ms/step - loss: 0.0242 - accuracy: 0.9927 - mean_io_u_1: 0.8329 - dice_coef: 0.6366 - dice_coef_necrotic: 0.5410 - dice_coef_edema: 0.7496 - dice_coef_enhancing: 0.6856


[0.024185582995414734,
 0.9926924109458923,
 0.8329079151153564,
 0.6365877985954285,
 0.5409764647483826,
 0.7495592832565308,
 0.6855665445327759]